In [4]:
import os, pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflowjs as tfjs

In [5]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Set seeds
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# Define parameters
batch_size = 32
height = 400
width = 400

init_train_ds = keras.preprocessing.image_dataset_from_directory(
    "drive/My Drive/CLASSIFY_BALANCED_BINARY",
    validation_split=0.2,
    image_size=(1000, 1000),
    subset="training",
    seed=1,
    batch_size=batch_size,
    label_mode="int",
    crop_to_aspect_ratio=True,
)

init_val_ds = keras.preprocessing.image_dataset_from_directory(
    "drive/My Drive/CLASSIFY_BALANCED_BINARY",
    validation_split=0.2,
    image_size=(1000, 1000),
    subset="validation",
    seed=1,
    batch_size=batch_size,
    label_mode="int",
    crop_to_aspect_ratio=True,
)

# Random crop
random_crop = keras.Sequential([keras.layers.RandomCrop(height, width, seed=1)])
train_ds = init_train_ds.map(lambda x, y: (random_crop(x), y))
# Center crop
center_crop = keras.Sequential([keras.layers.CenterCrop(height, width)])
val_ds = init_val_ds.map(lambda x, y: (center_crop(x), y))

#########################################################################################################################

base_model = keras.applications.Xception(
    include_top=False, weights="imagenet", input_shape=(height, width, 3)
)
# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(height, width, 3))
x = inputs

# Normalize
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode when we unfreeze the base model for fine-tuning, so we make sure that the base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)


#########################################################################################################################


model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=150,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.001, patience=10)
    ],
)

# Unfreeze the base_model. Note that it keeps running in inference mode since we passed `training=False` when calling it. This means that the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training we've done so far.
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.001, patience=10)
    ],
)

Found 1872 files belonging to 2 classes.
Using 1498 files for training.
Found 1872 files belonging to 2 classes.
Using 374 files for validation.
83697664/83683744 [==============================] - 2s 0us/step
Epoch 1/150
47/47 [==============================] - 159s 2s/step - loss: 0.6739 - accuracy: 0.5861 - val_loss: 0.6244 - val_accuracy: 0.6684
Epoch 2/150
47/47 [==============================] - 34s 632ms/step - loss: 0.6256 - accuracy: 0.6676 - val_loss: 0.6169 - val_accuracy: 0.6604
Epoch 3/150
47/47 [==============================] - 33s 626ms/step - loss: 0.6068 - accuracy: 0.6829 - val_loss: 0.5847 - val_accuracy: 0.6979
Epoch 4/150
47/47 [==============================] - 34s 636ms/step - loss: 0.6036 - accuracy: 0.6802 - val_loss: 0.5763 - val_accuracy: 0.7032
Epoch 5/150
47/47 [==============================] - 34s 643ms/step - loss: 0.5797 - accuracy: 0.7009 - val_loss: 0.5603 - val_accuracy: 0.7299
Epoch 6/150
47/47 [==============================] - 34s 636ms/step - lo

In [6]:
model.save("drive/My Drive/Models/BINARY")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
